In [169]:
import pandas as pd
import math
file = pd.read_csv('45826522P9.txt',header = None)

In [170]:
#回傳一個陣列，裡面有  除了第一列(也就是時間列)的所有value count
#呼叫時parameters放入要讀取的檔案
def countValues(file):
    file_header = list(file)
    unique_count = {}
    for i in range(1,len(file_header)):  #刻意不算時間欄位
        valueCounts = file[file_header[i]].value_counts()
        for j in range(len(valueCounts.index)):
            unique_count[valueCounts.index[j]] = valueCounts.values[j]
    return unique_count

#以下沒用到，以後要用可以參考
#unique_names = file[file_header[i]].value_counts().index.tolist() #算出個欄位unique值
#unique_count[valueCounts.indexs] = valueCounts.values
        

In [ ]:
# countValues(file)

In [171]:
countValuesList = countValues(file)

In [172]:
def activityCount(file):  #使用這個function請先確定資料是照日期排序，且第二欄位為TrafficSource
    sessionJumpCount = 0 #算session跳另一組的次數
    rowCount = 0
    for i in range(len(file.index)):
        if i==0: 
            rowCount =1 
            continue
        if file[2][i] == file[2][i-1]: #如果session連續
            rowCount += 1
        else:
            #print("換另一session")
            sessionJumpCount += 1
            rowCount +=1
            
    print("rowCount = " + str(rowCount))
    print("sessionJumpCount =" +str(sessionJumpCount))
    # print( math.log2(rowCount) + sessionJumpCount) #97
    return math.sqrt(rowCount)+sessionJumpCount

In [173]:
activityCount(file)

rowCount = 10173
sessionJumpCount =1045


1145.8612908900138

In [174]:
def purchaseActivityCount(countValuesList): #執行過countValues函數，把那個函數的return值丟進來才能算
    purchaseActivityDict = {}
    favCount = countValuesList['Fav']
    cartCount = countValuesList['Cart']
    purchaseCount = countValuesList['Purchase']
    purchaseCartConversionRate = purchaseCount/cartCount # 購物次數/cart次數
    favPurchaseConversionRate = favCount/purchaseCount # fav次數/購物次數
    #所以purchaseCartConversionRate*favCartConversionRate =  fav次數/cart次數
    favCartConversionRate = purchaseCartConversionRate*favPurchaseConversionRate # fav/Cart
    purchaseActivityDict.update({'favCount' : favCount,
                                'cartCount' : cartCount,
                                'purchaseCount' : purchaseCount,
                                'purchaseCartConversionRate' : purchaseCartConversionRate,
                                'favPurchaseConversionRate' : favPurchaseConversionRate,
                                'favCartConversionRate' : favCartConversionRate})
    return purchaseActivityDict

In [175]:
purchaseActivityCount(countValuesList)

{'favCount': 37,
 'cartCount': 142,
 'purchaseCount': 64,
 'purchaseCartConversionRate': 0.4507042253521127,
 'favPurchaseConversionRate': 0.578125,
 'favCartConversionRate': 0.2605633802816901}

In [12]:
# print(file[3][1])

ViewSalePage


In [ ]:
# for i, row in enumerate(file.values):
#     print(i)
#     print(type(row[0]))
#     print(file.values[i-1])

In [ ]:
# for i, row in enumerate(file.values):
#     #print((pd.to_datetime(row[0]) - pd.to_datetime(file.values[i-1][0])).total_seconds())
#     print(i)
#     if(i%3==0):
#         i+=2

In [74]:
# print(len(file.index))

10173


In [154]:
# txt = "8888 55 44"
# txt.split(" ")

['8888', '55', '44']

In [176]:
import datetime
def purchaseBeforeCount(file):
    purchase_segment_dict = {}
    a_purchase_segment_row = 0
    a_purchase_segment_product_list = {}
    a_purchase_segment_source_type = {}
    a_purchase_segment_SessionCount=0
    a_purchase_order = {}
    purchase_orders = []
    purchase_order_serial_number = 0 #訂單編號
    test=[]
    i=0
    while(i+1 != len(file.index)):
        for i, row in enumerate(file.values):

            if(row[3]!='Purchase'):
                a_purchase_segment_row += 1
                try:
                    a_purchase_segment_source_type["{}".format(row[4])] = a_purchase_segment_source_type["{}".format(row[4])]+1
                except:
                    a_purchase_segment_source_type.update({"{}".format(row[4]) : 1})
                try:
                    a_purchase_segment_product_list["{}".format(row[5])] = a_purchase_segment_product_list["{}".format(row[5])]+1 #該品項或商品的count+1
                except:
                    a_purchase_segment_product_list.update({"{}".format(row[5]) : 1})
                
            #算purchase裡，session的跳動  => 11231為跳四次 
                if(row[2]!=file.values[i-1][2]): # i-1為上一行
                    if((pd.to_datetime(row[0]) - pd.to_datetime(file.values[i-1][0])).total_seconds()>= 1800):
                        a_purchase_segment_SessionCount += 1 
                        
            
            #如果該筆資料為purchase，把之前計算之數值放入dict,並且全部重新計算
            elif(row[3]=='Purchase'):
                test.append(a_purchase_segment_SessionCount)
#                 print(a_purchase_segment_source_type)
#                 print(a_purchase_segment_product_list)
                
                #before_purchase_list.append(before_purchase_row)
                
                
                
                a_purchase_segment_source_type = {}
                a_purchase_segment_product_list = {}
                a_purchase_segment_row = 0
                a_purchase_segment_SessionCount = 0
                
                
                #Purchase單 計算
                if(row[0] == file.values[i-1][0]): #如果時間相同
                    try:
                        a_purchase_order["{}".format(row[5]).split(" ")[0]] = a_purchase_segment_product_list["{}".format(row[5]).split(" ")[0]]+1
                    except:
                        a_purchase_order.update({"{}".format(row[5]).split(" ")[0] : 1})
                    purchase_amount += float(row[5].split(" ")[1])
                    discount_amount += float(row[5].split(" ")[2])
                    a_purchase_order['purchase_amount'] = purchase_amount
                    a_purchase_order['discount_amount'] = discount_amount
                else:
                    if(a_purchase_order == {}):
                        a_purchase_order.update({"{}".format(row[5]).split(" ")[0] : 1})
                        a_purchase_order['purchase_amount'] = float(row[5].split(" ")[1])
                        a_purchase_order['discount_amount'] = float(row[5].split(" ")[2])
                    purchase_orders.append({'purchaseOrder{}'.format(purchase_order_serial_number) : a_purchase_order})
                    purchase_amount = float(row[5].split(" ")[1])
                    discount_amount = float(row[5].split(" ")[2])
                    purchase_order_serial_number += 1
                    a_purchase_order = {}
                    
    return test,purchase_orders



#     sessionJumpCount = 0 #算session跳另一組的次數
#     rowCount = 0
#     for i in range(len(file.index)):
#         if i==0: 
#             rowCount =1 
#             continue
#         if file[2][i] == file[2][i-1]: #如果session連續
#             rowCount += 1
#         else:
#             #print("換另一session")
#             sessionJumpCount += 1
#             rowCount +=1
            
#     print("rowCount = " + str(rowCount))
#     print("sessionJumpCount =" +str(sessionJumpCount))        
            
            
    #before_purchase_list.append(每個dict)        
    #return before_purchase_list

In [177]:
a = purchaseBeforeCount(file)
a

([45,
  45,
  0,
  0,
  3,
  0,
  24,
  0,
  0,
  17,
  0,
  10,
  0,
  0,
  32,
  0,
  5,
  0,
  0,
  0,
  18,
  4,
  20,
  0,
  47,
  9,
  7,
  0,
  32,
  15,
  9,
  0,
  11,
  21,
  5,
  13,
  0,
  28,
  0,
  55,
  0,
  33,
  0,
  16,
  0,
  24,
  0,
  0,
  2,
  0,
  35,
  0,
  9,
  0,
  26,
  0,
  13,
  0,
  18,
  0,
  42,
  0,
  0,
  0],
 [{'purchaseOrder0': {'4297994': 1,
    'purchase_amount': 2560.0,
    'discount_amount': 0.0}},
  {'purchaseOrder1': {'4305841': 1,
    'purchase_amount': 3450.0,
    'discount_amount': 0.0}},
  {'purchaseOrder2': {'4396756': 1,
    'purchase_amount': 10350.0,
    'discount_amount': 0.0,
    '4396777': 1}},
  {'purchaseOrder3': {'4286297': 1,
    'purchase_amount': 3160.0,
    'discount_amount': 0.0}},
  {'purchaseOrder4': {'4482455': 1,
    'purchase_amount': 7680.0,
    'discount_amount': 0.0}},
  {'purchaseOrder5': {'4422498': 1,
    'purchase_amount': 4720.0,
    'discount_amount': 0.0}},
  {'purchaseOrder6': {'4508699': 1,
    'purchase_amou

In [178]:
len(a)

2